In [86]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Reusable classes
import sys
sys.path.append("/home/jovyan/work/upload/ml-experiments/sc_wrangling/")

# Data path
import pathlib
FOLDER = pathlib.Path("/home/jovyan/work/upload/")
TABLE_PATH = FOLDER / 'data/pickle/df_columns_labels.pkl'

# Manage ML lifecycle
import mlflow
from mlflow import log_params, set_tags, log_metrics
MLFLOW_TRACKING_URI = 'http://srv.meaningfy.ws:8989'

# Data visualisation
import pandas as pd

# Vectorize algotrithm
from gensim.models import Word2Vec

# Random Forest Algorithm
from sklearn.ensemble import RandomForestClassifier as RFC
# :n_estimators: The number of trees in the forest
N_ESTIMATORS = 17

# Intermediate steps of the pipeline must be ‘transforms’,
# that is, they must implement fit and transform methods.
from sklearn.pipeline import Pipeline

# Splitting data into train and test
from sklearn.model_selection import train_test_split
RANDOM_STATE = 42
# Where "train" contains 70% of data and "test" - 30%
TEST_SIZE = 0.3
SHUFFLE = True


# Mean embedding function
from mean_vectorizer import MeanEmbeddingVectorizer

# Metrics Evaluation Methods
from evaluation_metrics import model_evaluation_metrics

# Transform DataFrame to dictionary
from dictionary_transformation import series_pair_to_dict


In [17]:
# Read data
df = pd.read_pickle(TABLE_PATH)
df.head()

,Concatenated Data (clean),Category,Subcategory,Type of measure,Target groups,One person or microenterprises|Self-employed|Solo-self-employed,SMEs|Sector specific set of companies,One person or microenterprises|SMEs,Employees in standard employment|Other groups of workers,Employees in standard employment|Larger corporations,...,Children (minors)|Disabled|Older citizens|Parents|SMEs|Single parents|Single parents in employment|The COVID-19 risk group|Workers in care facilities|Workers in essential services,Employees in standard employment|Workers in care facilities|Workers in non-standard forms of employment,Companies providing essential services|Workers in essential services,Contractors of a company,Other businesses|Unemployed,Seasonal workers|Workers in non-standard forms of employment,Employees in standard employment|Particular professions,Businesses,Citizens,Workers
0,"[hardship, case, fund, safety, net, selfemploy...",2,12,0,One person or microenterprises|Self-employed|S...,1,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
1,"[state, support, tourism, access, financeas, t...",0,0,0,SMEs|Sector specific set of companies,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,"[bank, guarantees, smes, oneperson, enterprise...",0,0,0,One person or microenterprises|SMEs,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,"[emergency, measures, relating, shorttime, wor...",6,14,0,Employees in standard employment|Other groups ...,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1
4,"[airbus, agreement, making, unworked, hours, p...",3,7,2,Employees in standard employment|Larger corpor...,0,0,0,0,1,...,0,0,0,0,0,0,0,1,1,1


In [18]:
# Independent data
columns = df['Concatenated Data (clean)']
# Label data
category = df['Category']

## word2vec model

In [19]:
# Insert our data and set minimal word count to 10, and size of each word to 300 vectors
model = Word2Vec.load('/home/jovyan/work/Dan/data/pwdb/word2vec/df.model')
w2v_dict = {w: vec for w, vec in zip(model.wv.index2word, model.wv.syn0)}

In [140]:
# Use pipes to implement steps of fit and transform method
rfc_w2v = Pipeline([
    # Add the words we want to mean
    ("word2vec vectorizer", MeanEmbeddingVectorizer(w2v_dict)),
    ("Random Forest", RFC(n_estimators=N_ESTIMATORS))])

## Train SVM Model

In [134]:
# Split data into test and train sets
X_train, X_test, y_train, y_test = train_test_split(
    columns, category, random_state=RANDOM_STATE, test_size=TEST_SIZE, shuffle=SHUFFLE)

In [141]:
# Fit Random Forest model with out train data
rfc_w2v.fit(X_train, y_train)

Pipeline(steps=[('word2vec vectorizer',
                 <mean_vectorizer.MeanEmbeddingVectorizer object at 0x7fb886972e50>),
                ('Random Forest',
                 RandomForestClassifier(n_estimators=17, random_state=32))])

In [142]:
# Model Evaluation
rfc_w2v.score(X_train, y_train)

0.9954128440366973

In [143]:
# Random Forest prediction based on test data
prediction = rfc_w2v.predict(X_test)
prediction

array([0, 6, 0, 2, 3, 0, 0, 0, 0, 4, 4, 2, 0, 0, 0, 4, 2, 3, 3, 9, 6, 4,
       0, 0, 0, 4, 0, 8, 2, 3, 2, 2, 4, 7, 0, 3, 7, 7, 4, 0, 7, 7, 7, 4,
       4, 7, 4, 3, 0, 3, 3, 3, 6, 3, 3, 3, 0, 4, 0, 4, 0, 0, 0, 4, 2, 0,
       0, 6, 3, 4, 3, 6, 0, 0, 3, 0, 7, 2, 2, 4, 2, 5, 0, 0, 0, 0, 0, 3,
       3, 6, 4, 0, 0, 0, 5, 3, 0, 0, 0, 0, 4, 8, 2, 3, 2, 0, 4, 3, 0, 3,
       4, 0, 2, 0, 2, 2, 6, 3, 0, 5, 0, 4, 0, 0, 0, 4, 6, 0, 5, 3, 0, 0,
       7, 0, 0, 3, 0, 0, 5, 5, 3, 4, 6, 3, 5, 0, 7, 0, 2, 6, 0, 2, 5, 2,
       2, 4, 0, 0, 5, 0, 3, 3, 5, 2, 2, 0, 5, 2, 2, 0, 2, 3, 3, 2, 9, 0,
       4, 0, 6, 0, 0, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 3, 2, 0, 3, 5, 2, 9,
       4, 3, 3, 2, 0, 6, 7, 6, 3, 2, 0, 3, 6, 2, 0, 3, 0, 0, 4, 0, 0, 0,
       7, 0, 0, 3, 3, 0, 5, 4, 5, 3, 2, 0, 4, 0, 6, 4, 4, 6, 2, 0, 0, 2,
       4, 4, 0, 0, 0, 3, 4, 0, 0, 5, 3, 2, 3, 3, 3, 0, 7, 0, 2, 4, 4, 3,
       3, 0, 4, 4, 0, 4, 0, 0, 3, 3, 2, 2, 0, 5, 7, 0, 2])

## Evaluation

In [144]:
# View evaluation metrics table
evaluation = model_evaluation_metrics(y_test, prediction)
evaluation

,Metric Labels,Metric values
0,Accuracy,0.462633
1,Precision,0.306628
2,Recall,0.314060
3,F1-Score,0.301162
4,Mean Absolute Error,1.889680
5,Mean Squared Error,8.950178


In [145]:
# Transform columns data to dict to use in MLFlow
transformation = series_pair_to_dict(evaluation, 'Metric Labels', 'Metric values')
transformation

{'Accuracy': 0.4626334519572954,
 'Precision': 0.30662842551430386,
 'Recall': 0.31406032240392023,
 'F1-Score': 0.30116178432123053,
 'Mean Absolute Error': 1.8896797153024911,
 'Mean Squared Error': 8.95017793594306}

In [146]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="Category (Random Forest, Word2Vec)")

with mlflow.start_run():

    parameters = {"Language model": 'word2vec',
                  "Random state": RANDOM_STATE,
                  "Test size": TEST_SIZE,
                  "Shuffle": SHUFFLE,
                  "n_estimator": N_ESTIMATORS}
    log_params(parameters)
    log_metrics(transformation)